In [ ]:
import numpy as np
import os
from pathlib import Path
import shutil
from skimage import io
from tqdm import tqdm

In [ ]:
# directory containing simulation and experiment video subdirectories
SRC_DIR = '/mnt/labData/joshData/phiVAE/raw_data' # CHANGE THIS!!!
# directory to save datasets as subdirectories
DST_DIR = '/mnt/labData/joshData/phiVAE/datasets' # CHANGE THIS!!!

In [ ]:
def complexify_frame(phi, inscale='tif', savepath=None):
    '''
    Converts phase field into array of unit complex numbers and returns or saves as TIFF

    Parameters
    ----------
    phi: 64 x 64 array-like, 2D phase field
    scale: {'radian', 'cycle', 'tif'}, absolute range of angles
        'radian': [-pi, pi)
        'cycle': [0, 1]
        'tif': [0, 255]
    savepath: str, path to save array of unit complex numbers

    Returns
    -------
    x: 2 x 64 x 64 np.ndarray, stack([cos(phi), sin(phi)]) in tif scale

    '''
    # rescale angles to radians
    if inscale == 'tif': phases = np.array(phi)/255.0*2.0*np.pi-np.pi
    elif inscale == 'cycle': phases = np.array(phi)*2.0*np.pi-np.pi
    elif inscale == 'radian': phases = np.array(phi)
    else: raise ValueError('Unknown input scale: {}'.format(inscale))
    
    # represent angle as unit complex number
    x = np.stack((np.cos(phases), np.sin(phases)), dtype=np.single)
    x = np.rint((x+1.0)/2.0*255.0).astype('uint8')
    condition = (np.array([phases<0]*2)*np.stack((np.zeros_like(phases), x[1]==128))).astype(bool)
    x = np.where(condition, 127, x) # deals with a boundary case where -pi is mapped to pi
    # with this correction phi can be reconstructed unambiguously from x

    if savepath != None: io.imsave(savepath, x)
    else: return x

In [ ]:
# make vanishing-field reference sample
ruler_path = os.path.join(SRC_DIR, 'ruler.tif')
complexify_frame(128*np.ones((64, 64)), savepath=ruler_path)

In [ ]:
def complexify_stack(src, dst, ruler_path=None):
    '''
    Reads TIFF stack encoding phase field and creates complex-valued dataset

    Parameters
    ----------
    src: str, path to input TIFF stack
    dst: str, path to output directory
    ruler_path: str, path to vanishing-field reference sample
    '''
    vid = io.imread(src)
    n = len(vid)
    Path(dst).mkdir(parents=True)
    fmt = '{:0' + str(len(str(n))) + 'd}'

    if ruler_path != None:
        shutil.copyfile(ruler_path, os.path.join(dst, fmt.format(n)+'.tif'))
    else: pass

    for i in range(n):
        complexify_frame(vid[i], savepath=os.path.join(dst, fmt.format(i)+'.tif'))

In [ ]:
def complexify_multi(src, dst, ruler_path=None):
    '''
    Creates complex-valued dataset for all TIFFs in directory

    Parameters
    ----------
    src: str, path to input directory
    dst: str, path to output directory
    ruler_path: str, path to vanishing-field reference ruler sample
    '''
    vids = np.sort([i for i in os.listdir(src) if '.tif' in i])
    for vid in tqdm(vids):
        complexify_stack(os.path.join(src, vid),
                         os.path.join(dst, vid.replace('.tif', '')),
                         ruler_path=ruler_path)

In [ ]:
# make sim_tstep_vary datasets
complexify_multi(os.path.join(SRC_DIR, 'sim_tstep_vary'),
                 os.path.join(DST_DIR, 'sim_tstep_vary'),
                 ruler_path=ruler_path)

In [ ]:
# make sim_noise_vary datasets
complexify_multi(os.path.join(SRC_DIR, 'sim_noise_vary'),
                 os.path.join(DST_DIR, 'sim_noise_vary'),
                 ruler_path=ruler_path)

In [ ]:
# make sim_regime_vary datasets
complexify_multi(os.path.join(SRC_DIR, 'sim_regime_vary'),
                 os.path.join(DST_DIR, 'sim_regime_vary'),
                 ruler_path=ruler_path)

In [ ]:
# make exp_atp_vary datasets
complexify_multi(os.path.join(SRC_DIR, 'exp_atp_vary'),
                 os.path.join(DST_DIR, 'exp_atp_vary'),
                 ruler_path=ruler_path)

In [ ]:
# make exp_orderp_vary datasets
complexify_multi(os.path.join(SRC_DIR, 'exp_orderp_vary'),
                 os.path.join(DST_DIR, 'exp_orderp_vary'),
                 ruler_path=ruler_path)